# How does SHA-256 work?
### Simple hasing algorithm 256

In [75]:
from hashlib import sha256

m = sha256()
print(f"Hash type:\t{m.name}")
print(f"Void digest:\t{m.hexdigest()}")
print(f"Digest size:\t{m.digest_size}")

string = "Hello World!"
m.update(string.encode())
print(f"\nString:\t{string}")
print(f"Digest:\t{m.hexdigest()}")


Hash type:	sha256
Void digest:	e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
Digest size:	32

String:	Hello World!
Digest:	7f83b1657ff1fc53b92dc18148a1d65dfc2d4b1fa3d677284addd200126d9069


## Atomic bit level operations

1. SHR
2. ROTR
3. XOR
4. SUM

In [127]:
from IPython.display import clear_output
from bitarray import bitarray
from time import sleep

### 1. SHR
Shift right

In [77]:
def shr(bits, shifts):
    for _ in range(shifts):
        bits.pop()
        bits.insert(0, 0)
    return bits

a = bitarray(endian='big')
a.frombytes(b'\xff\x00\xff\x00')

print("{:>8} {:>32}".format("A:", a.to01()))
for i in range(32):
    print("\r{:>8} {:>32}".format("SHR "+str(i+1)+":", shr(a,1).to01()), end="")
    sleep(0.2)
del a, i

      A: 11111111000000001111111100000000
 SHR 32: 00000000000000000000000000000000

### 2. ROTR
Rotation right

In [78]:
def rotr(bits, shifts):
    for _ in range(shifts):
        bits.insert(0,bits.pop())
    return bits

a = bitarray(endian='big')
a.frombytes(b'\xff\x00\xff\x00')

print("{:>8} {:>32}".format("A:", a.to01()))
for i in range(32):
    print("\r{:>8} {:>32}".format("ROTR "+str(i+1)+":", rotr(a,1).to01()), end="")
    sleep(0.2)
del a, i

      A: 11111111000000001111111100000000
ROTR 32: 11111111000000001111111100000000

### 3. XOR
Exclusive OR



| A | B | XOR |
|---|---|-----|
| 0 | 0 | 0   |
| 0 | 1 | 1   |
| 1 | 0 | 1   |
| 1 | 1 | 0   |   



In [79]:
def xor(a, b):
    return a^b

a = bitarray(endian='big')
b = bitarray(endian='big')
a.frombytes(b'\xff\x00\xff\x00')
b.frombytes(b'\x00\xff\xff\x00')

print("{:>8} {:>32}".format("A:", a.to01()))
print("{:>8} {:>32} XOR".format("B:", b.to01()))
print("{:>8} {:>32}".format("", "-"*32))
for i in range(32):
    print("\r{:>8} {:>32}".format("", xor(a,b).to01()[32-1-i:]), end="")
    sleep(0.2)
del a, b, i


      A: 11111111000000001111111100000000
      B: 00000000111111111111111100000000 XOR
         --------------------------------
         11111111111111110000000000000000

### 4. SUM
Binary addition <br>
Mod 32, ignoring the carry out that will overflow the 32 bit.

In [80]:
def add(a, b):
    result = bitarray(endian='big')
    carry = 0
    for i in range(32-1, -1, -1):
        result.insert(0,(a[i]^b[i]^carry))
        carry = 1 if (a[i] + b[i] + carry) > 1 else 0
    return result


a = bitarray(endian='big')
b = bitarray(endian='big')
a.frombytes(b'\x80\x30\xff\xff')
b.frombytes(b'\x80\x50\x00\x00')

print("{:>8} {:>32}".format("A:", a.to01()))
print("{:>8} {:>32} +".format("B:", b.to01()))
print("{:>8} {:>32}".format("", "-"*32))
out = add(a,b)
for i in range(32):
    print("\r{:>8} {:>32}".format("", out.to01()[32-1-i:]), end="")
    sleep(0.2)
del a, b, c, m, i

      A: 10000000001100001111111111111111
      B: 10000000010100000000000000000000 +
         --------------------------------
         00000000100000001111111111111111

***
## Compound operations

1. $\sigma0$
2. $\sigma1$
3. $\Sigma0$
4. $\Sigma1$


### 1. $\sigma0$

In [113]:
def sigma0(a, b, c):
    rotr(a, 7)
    rotr(b, 18)
    shr(c, 3)
    return xor(xor(a,b),c)

origin = bitarray(endian='big')
origin.frombytes(b'\x00\x00\x3f\xff')

a = origin.copy()
b = origin.copy()
c = origin.copy()

result = sigma0(a, b, c)

print("{:>8} {:>32}".format("Origin:", origin.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("ROTR  7:", a.to01()))
print("{:>8} {:>32} XOR".format("ROTR 18:", b.to01()))
print("{:>8} {:>32} XOR".format("SHR  7:", c.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("\u03C30:", result.to01()))

del a, b, c, result, origin



 Origin: 00000000000000000011111111111111
         --------------------------------
ROTR  7: 11111110000000000000000001111111
ROTR 18: 00001111111111111100000000000000 XOR
 SHR  7: 00000000000000000000011111111111 XOR
         --------------------------------
     σ0: 11110001111111111100011110000000


### 1. $\sigma1$

In [114]:
def sigma1(a, b, c):
    rotr(a, 17)
    rotr(b, 19)
    shr(c, 10)
    return xor(xor(a,b),c)

origin = bitarray(endian='big')
origin.frombytes(b'\x00\x00\x3f\xff')

a = origin.copy()
b = origin.copy()
c = origin.copy()

result = sigma1(a, b, c)

print("{:>8} {:>32}".format("Origin:", origin.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("ROTR 17:", a.to01()))
print("{:>8} {:>32} XOR".format("ROTR 19:", b.to01()))
print("{:>8} {:>32} XOR".format("SHR 10:", c.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("\u03C31:", result.to01()))

del a, b, c, result, origin


 Origin: 00000000000000000011111111111111
         --------------------------------
ROTR 17: 00011111111111111000000000000000
ROTR 19: 00000111111111111110000000000000 XOR
 SHR 10: 00000000000000000000000000001111 XOR
         --------------------------------
     σ1: 00011000000000000110000000001111


### 3. $\Sigma0$

In [115]:
def Sigma0(a, b, c):
    rotr(a, 2)
    rotr(b, 13)
    rotr(c, 22)
    return xor(xor(a,b),c)

origin = bitarray(endian='big')
origin.frombytes(b'\x00\x00\x3f\xff')

a = origin.copy()
b = origin.copy()
c = origin.copy()

result = Sigma0(a, b, c)

print("{:>8} {:>32}".format("Origin:", origin.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("ROTR  2:", a.to01()))
print("{:>8} {:>32} XOR".format("ROTR 13:", b.to01()))
print("{:>8} {:>32} XOR".format("ROTR 22:", c.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("\u03A30:", result.to01()))

del a, b, c, result, origin


 Origin: 00000000000000000011111111111111
         --------------------------------
ROTR  2: 11000000000000000000111111111111
ROTR 13: 11111111111110000000000000000001 XOR
ROTR 22: 00000000111111111111110000000000 XOR
         --------------------------------
     Σ0: 00111111000001111111001111111110


### 4. $\Sigma1$

In [116]:
def Sigma1(a, b, c):
    rotr(a, 6)
    rotr(b, 11)
    rotr(c, 25)
    return xor(xor(a,b),c)

origin = bitarray(endian='big')
origin.frombytes(b'\x00\x00\x3f\xff')

a = origin.copy()
b = origin.copy()
c = origin.copy()

result = Sigma1(a, b, c)

print("{:>8} {:>32}".format("Origin:", origin.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("ROTR  6:", a.to01()))
print("{:>8} {:>32} XOR".format("ROTR 11:", b.to01()))
print("{:>8} {:>32} XOR".format("ROTR 25:", c.to01()))
print("{:>8} {:>32}".format("", "-"*32))
print("{:>8} {:>32}".format("\u03A31:", result.to01()))

del a, b, c, result, origin

 Origin: 00000000000000000011111111111111
         --------------------------------
ROTR  6: 11111100000000000000000011111111
ROTR 11: 11111111111000000000000000000111 XOR
ROTR 25: 00000000000111111111111110000000 XOR
         --------------------------------
     Σ1: 00000011111111111111111101111000


### 5. Choice

In [143]:
def choice(choice, a, b):
    result = choice.copy()
    for i in range(32):
        if choice[i] == 1:
            result[i] = a[i]
        elif choice[i] == 0:
            result[i] = b[i]
    return result

a = bitarray(endian='big')
b = bitarray(endian='big')
c = bitarray(endian='big')

a.frombytes(b'\x00\xff\x00\xff')
b.frombytes(b'\x00\x00\xff\xff')
c.frombytes(b'\xff\xff\x00\x00')

result = choice(a, b, c)

for i in range(32):

    print(" "*(41-(i+1))+"\u21D3")
    print("{:>8} {:>32}".format("A:", a.to01()))
    print("{:>8} {:>32} {}".format("B:", b.to01(), "" if a[i]==1 else "\u21E6"))
    print("{:>8} {:>32} {}".format("C:", c.to01(), "" if a[i]==0 else "\u21E6"))
    print("{:>8} {:>32}".format("", "-"*32))
    print("{:>8} {:>32}".format("OUT:", result.to01()[32-(i+1):]))
    
    sleep(0.2)
    clear_output(wait=True)
    

del a, b, c, result

         ⇓
      A: 00000000111111110000000011111111
      B: 00000000000000001111111111111111 
      C: 11111111111111110000000000000000 ⇦
         --------------------------------
    OUT: 11111111000000000000000011111111
